In [6]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import optuna
import gc

In [2]:
!nvidia-smi

Sun Nov 14 14:59:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN RTX    On   | 00000000:09:00.0 Off |                  N/A |
| 41%   47C    P8    29W / 280W |      6MiB / 24220MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA TITAN RTX    On   | 00000000:41:00.0  On |                  N/A |
| 41%   

In [3]:
train_x = dd.read_csv('../input/xgtrain.csv')
test_x = dd.read_csv('../input/xgval.csv')

In [4]:
train_y = train_x['target']
train_x = train_x[train_x.columns.difference(['target'])]

test_y = test_x['target']
test_x = test_x[test_x.columns.difference(['target'])]

In [8]:
train_x = train_x.replace([np.inf, -np.inf], np.nan)
test_x = test_x.replace([np.inf, -np.inf], np.nan)

In [9]:
cluster = LocalCUDACluster(n_workers=2)
client = Client(cluster)

distributed.preloading - INFO - Import preload module: dask_cuda.initialize
distributed.preloading - INFO - Import preload module: dask_cuda.initialize


In [10]:
dtrain = xgb.dask.DaskDMatrix(client, train_x, train_y)

In [11]:
dtest = xgb.dask.DaskDMatrix(client, test_x, test_y)

In [12]:
num_round = 1000

In [13]:
def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['binary:logistic']), 
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_categorical('max_depth', [3,5,7,9,11,13,15,17,20]),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['logloss']),

    }

    output = xgb.dask.train(client, params, dtrain, num_round)
    
    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, dtest)
    
    predictions = predictions.compute()

    roc = roc_auc_score(test_y, predictions)
    
    return roc

In [14]:
study = optuna.create_study(direction='maximize')

[I 2021-11-14 15:01:40,681] A new study created in memory with name: no-name-64bf40ca-f6c9-4d64-82c0-363b969913bd


In [15]:
%%time
study.optimize(objective, n_trials=2)

[15:01:41] task [xgboost.dask]:tcp://192.168.1.200:46877 got new rank 0
[15:01:41] task [xgboost.dask]:tcp://192.168.1.200:35873 got new rank 1
[I 2021-11-14 15:05:53,304] Trial 0 finished with value: 0.7900731755704626 and parameters: {'objective': 'binary:logistic', 'tree_method': 'gpu_hist', 'lambda': 0.041089615489132994, 'alpha': 0.00494036533080873, 'colsample_bytree': 0.9035315483013782, 'subsample': 0.597568483224114, 'learning_rate': 0.025070904873936418, 'max_depth': 20, 'min_child_weight': 30, 'eval_metric': 'logloss'}. Best is trial 0 with value: 0.7900731755704626.
[15:05:53] task [xgboost.dask]:tcp://192.168.1.200:35873 got new rank 0
[15:05:53] task [xgboost.dask]:tcp://192.168.1.200:46877 got new rank 1
[I 2021-11-14 15:09:50,526] Trial 1 finished with value: 0.7939551924492353 and parameters: {'objective': 'binary:logistic', 'tree_method': 'gpu_hist', 'lambda': 0.018860800189022853, 'alpha': 0.817203366904566, 'colsample_bytree': 0.847225587363762, 'subsample': 0.49658

CPU times: user 15.7 s, sys: 4.34 s, total: 20.1 s
Wall time: 8min 8s


In [16]:
%%time
study.optimize(objective, n_trials=50)

[15:09:50] task [xgboost.dask]:tcp://192.168.1.200:35873 got new rank 0
[15:09:50] task [xgboost.dask]:tcp://192.168.1.200:46877 got new rank 1
[I 2021-11-14 15:10:26,968] Trial 2 finished with value: 0.7482935556144774 and parameters: {'objective': 'binary:logistic', 'tree_method': 'gpu_hist', 'lambda': 0.004544023309469089, 'alpha': 0.004342151485300922, 'colsample_bytree': 0.6045626423557863, 'subsample': 0.5213778810890927, 'learning_rate': 0.0011009658611694805, 'max_depth': 3, 'min_child_weight': 262, 'eval_metric': 'logloss'}. Best is trial 1 with value: 0.7939551924492353.
[15:10:26] task [xgboost.dask]:tcp://192.168.1.200:35873 got new rank 0
[15:10:26] task [xgboost.dask]:tcp://192.168.1.200:46877 got new rank 1
[I 2021-11-14 15:11:43,607] Trial 3 finished with value: 0.7747693759389768 and parameters: {'objective': 'binary:logistic', 'tree_method': 'gpu_hist', 'lambda': 7.006770996307279, 'alpha': 0.016574919358730015, 'colsample_bytree': 0.6809303154339557, 'subsample': 0.7

CPU times: user 2min 18s, sys: 52.7 s, total: 3min 11s
Wall time: 1h 16min 41s


In [17]:
study.best_trial.params

{'objective': 'binary:logistic',
 'tree_method': 'gpu_hist',
 'lambda': 0.008416757023168712,
 'alpha': 0.017666476922893744,
 'colsample_bytree': 0.8709207057046353,
 'subsample': 0.8281737773756209,
 'learning_rate': 0.02395255427503995,
 'max_depth': 15,
 'min_child_weight': 155,
 'eval_metric': 'logloss'}

In [18]:
%%time
study.optimize(objective, n_trials=100)

[16:34:01] task [xgboost.dask]:tcp://192.168.1.200:35873 got new rank 0
[16:34:01] task [xgboost.dask]:tcp://192.168.1.200:46877 got new rank 1
[I 2021-11-14 16:35:10,864] Trial 52 finished with value: 0.796962416921279 and parameters: {'objective': 'binary:logistic', 'tree_method': 'gpu_hist', 'lambda': 0.0033566749131016967, 'alpha': 6.545584979065341, 'colsample_bytree': 0.3006188592741036, 'subsample': 0.9926157227727729, 'learning_rate': 0.0289743976219045, 'max_depth': 15, 'min_child_weight': 233, 'eval_metric': 'logloss'}. Best is trial 6 with value: 0.7977266704420776.
[16:35:10] task [xgboost.dask]:tcp://192.168.1.200:35873 got new rank 0
[16:35:10] task [xgboost.dask]:tcp://192.168.1.200:46877 got new rank 1
[I 2021-11-14 16:36:20,222] Trial 53 finished with value: 0.7966541944162749 and parameters: {'objective': 'binary:logistic', 'tree_method': 'gpu_hist', 'lambda': 0.0010339640797771433, 'alpha': 2.7998962935090987, 'colsample_bytree': 0.379013070750305, 'subsample': 0.971

CPU times: user 4min 45s, sys: 1min 37s, total: 6min 22s
Wall time: 2h 28min 6s
